Henter ut artikler og ingressen fra VG-artikler i Norsk Aviskorpus

Artikler uten Ingress eller innhold (ofte bak betalingsmur) beholdes ikke

URL, dato og kategorier beholdes og

In [215]:
import tarfile
import glob
import os

import xml.etree.ElementTree as ET

import tqdm
import pandas as pd


In [244]:
for nak_file in glob.glob("data/*.tar"):
    print(nak_file)
    with tarfile.open(nak_file, "r") as f:
        f.extractall("data/extracted/")

data\nak_2018.tar
data\nak_2019.tar


In [245]:
for vg_nak in glob.glob("data/extracted/*/vg-*-nob.tar.gz"):
    with tarfile.open(vg_nak, "r") as f:
        f.extractall("vg_nob_nak")

In [247]:
vg_articles = glob.glob("vg_nob_nak/*/VG*.xml")
print(f"{len(vg_articles)} VG-artikler funnet")

46490 VG-artikler funnet


In [248]:
def extract_from_xml(xml_tree):
    article = ""
    title = ""
    url = ""
    date = ""
    classes = ""

    div_elms = xml_tree.findall("*//div")
    for elm in div_elms:
        div_type = elm.attrib.get("type")
        if div_type == "ingress":
            ingress = elm.text.strip() if elm.text else None
        elif div_type == "text":
            sentences = elm.findall("p")
            article += " ".join(sentence.text.strip() for sentence in sentences)
        elif div_type == "title" and elm.attrib.get("level") == "1":
            title = elm.text.strip()

    if not article or not ingress:
        return None

    attr_elms = xml_tree.findall("*//attribute")
    for elm in attr_elms:
        attr_name = elm.attrib.get("name")
        if attr_name == "url":
            url = elm.attrib.get("value")
        elif attr_name == "date":
            date = elm.attrib.get("value")
        elif attr_name == "class1":
            classes = elm.attrib.get("value")


    return {"title": title, "url": url, "published": date, "classes": classes, "article": article, "ingress": ingress}


In [251]:
vg_summarization_dataset = pd.DataFrame(columns=["title", "url", "published", "classes", "article", "ingress"])

for vg_article in tqdm.tqdm(vg_articles[]):
    with open(vg_article, "r", encoding="utf-8") as xml_file:
        xml_tree = ET.fromstring(xml_file.read())
        extracted = extract_from_xml(xml_tree)
        if extracted:
            vg_summarization_dataset.loc[len(vg_summarization_dataset)] = extracted


100%|██████████| 90/90 [00:00<00:00, 114.70it/s]


In [252]:
vg_summarization_dataset

,title,url,published,classes,article,ingress
0,Illsint elgokse jaget mann opp i tre: - Truende,http://www.vg.no/nyheter/innenriks/i/4qKvj6/il...,2019-12-29 17:58,"nyheter,innenriks",Klokken 9.30 ringte det i nødtelefonen hos pol...,En mann i 60-åra så seg nødt til å klatre opp ...
1,Islandsk målmaskin skjøt gullet hjem: Vil vinn...,http://www.vg.no/sport/haandball/i/mRna64/isla...,2019-12-29 20:23,"sport,haandball",– Jeg vil veldig gjerne vinne trippelt før jeg...,OSLO SPEKTRUM (VG) (Haslum-Elverum 33 – 35) Ch...
2,Iversen reiser hjem fra Tour de Ski: - Tungt å...,http://www.vg.no/sport/langrenn/i/lAWbBA/ivers...,2019-12-29 16:25,"sport,langrenn",Skiforbundet skriver i en melding at han reise...,LENZERHEIDE (VG) Emil Iversen (28) må reise hj...
3,Joe Biden om riksrettssaken: – Ikke rettslig g...,http://www.vg.no/nyheter/utenriks/i/kJLb1L/joe...,2019-12-29 07:23,"nyheter,utenriks",– Jeg vil hedre det Kongressen rettmessig ber ...,Tidligere visepresident Joe Biden fastholder a...
4,Kjempebrøler av keeper da Arsenal raknet i Art...,http://www.vg.no/sport/fotball/i/2G0Kna/kjempe...,2019-12-29 18:58,"sport,fotball",Tyske Bernd Leno feilberegnet innsvingeren fra...,(Arsenal - Chelsea 1 – 2) Arsenal så ut til å ...
...,...,...,...,...,...,...
78,Michelle Williams (39) er gravid – og forlovet...,http://www.vg.no/rampelys/i/Jo2Gm4/michelle-wi...,2019-12-31 08:11,rampelys,"Nyheten om at Michelle Williams er gravid, bek...",«Dawson’s Creek»-skuespilleren har forlovet se...
79,Nyttårstrafikken: Sterk vind og glatte veier f...,http://www.vg.no/nyheter/innenriks/i/6jvdE3/ny...,2019-12-31 08:20,"nyheter,innenriks",Tirsdag morgen melder flere politidistrikt om ...,2019 ser ut til å bli avsluttet med mye vind o...
80,Raus med skrytet,http://www.vg.no/nyheter/meninger/i/pLEb7W/rau...,2019-12-31 06:31,"nyheter,meninger",På årets siste dag er det mange å ta farvel me...,Monica Mæland mener kommunereformen står til t...
81,Skogbrannene i Australia: Setter inn militærhe...,http://www.vg.no/nyheter/utenriks/i/pLEmvW/sko...,2019-12-31 07:49,"nyheter,utenriks","I Mallacoota, sør i delstaten Victoria, har fo...",SYDNEY/OSLO (VG) Skogbrannene i Australia fort...


In [238]:
vg_summarization_dataset.to_csv("test.csv")